In [ ]:
import os 
import sys 
import json 
from pathlib import Path 
from functools import cache
from itertools import product

cur_path = os.path.abspath(".")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.pagination import ShallowStrategy

from utils.utils import ddf, remove_prefix, load_subgraph, remove_keys
sg, bs = load_subgraph()
from collections import OrderedDict

# See UI logic in this directory for reference 
# https://github.com/BeanstalkFarms/Beanstalk-UI/tree/master/src/components/Analytics/Silo
beanstalk_addr = '0xC1E088fC1323b20BCBee9bd1B9fC9546db5624C5'
token_addrs = OrderedDict(dict(
    bean='0xBEA0000029AD1c77D3d5D23Ba2D8893dB9d1Efab'.lower(), 
    bean_3crv='0xc9C32cd16Bf7eFB85Ff14e0c8603cc90F6F2eE49'.lower(), 
    ur_bean='0x1BEA0050E63e05FBb5D8BA2f10cf5800B6224449'.lower(), 
    ur_bean_3crv='0x1BEA3CcD22F4EBd3d37d731BA31Eeca95713716D'.lower(), 
))
asset_ids = [f"{beanstalk_addr.lower()}-{token_addr.lower()}" for token_addr in token_addrs.values()]
queries = [
    bs.Query.siloAssetHourlySnapshots(
        first=10000, orderBy="season", orderDirection="asc",
        where={"season_gte": 6074, "siloAsset": aid}
    )
    for aid in asset_ids 
]
df_snaps_raw = pd.concat([
    sg.query_df(
        [
            q.id, 
            q.season, 
            q.timestamp, 
            q.totalDepositedAmount, 
            q.totalDepositedBDV, 
        ],
        pagination_strategy=ShallowStrategy
    )
    for q in queries
])

from collections import defaultdict

df_dep = df_snaps_raw.copy()
df_dep = remove_prefix(df_dep, 'siloAssetHourlySnapshots_').sort_values('timestamp').reset_index(drop=True)
df_dep['token'] = df_dep['id'].apply(lambda v: v.split('-')[1])
# df_dep = df_dep.groupby(['season', 'token']).last().reset_index()
# seasons = sorted(df_dep.season.unique())
# tokens = df_dep.token.unique()
# records = []
token_addr_inv = {v: k for k, v in token_addrs.items()}
# token_mult = OrderedDict(dict(
#     bean=1e-6, 
#     bean_3crv=1e-18, 
#     ur_bean=1e-6, 
#     ur_bean_3crv=1e-12, 
# ))
# last_value = defaultdict(float)
# for s, t in product(seasons, tokens):
#     metrics = {}
#     for m in ['totalDepositedAmount', 'totalDepositedBDV']: 
#         v = df_dep.loc[(df_dep.season == s) & (df_dep.token == t)][m]
#         if len(v): 
#             v = v.values[0]
#         else: 
#             v = last_value.get((t,m), 0)
#         v = v * token_mult[token_addr_inv[t]]
#         last_value[(t,m)] = v
#         metrics[m] = v
#     records.append({
#         "season": s, 
#         "token": t, 
#         **metrics 
#     })
# df_dep = pd.DataFrame(records)

df_dep['token_name'] = df_dep.token.apply(lambda v: token_addr_inv[v])

df_dep = df_dep.loc[df_dep.token_name == 'bean']
ddf(df_dep.tail(10))

df_dep = df_dep.loc[df_dep.token_name == 'bean']
alt.Chart(df_dep).mark_line().encode(
    x=alt.X("season:O"), 
    y=alt.Y("totalDepositedAmount:Q"), 
    color=alt.Color("token_name:N"), 
    tooltip=alt.Tooltip("totalDepositedAmount:Q")
).properties(width=700)
# query SeasonalDepositedUnripeBeans {
#   siloAssetDailySnapshots(
#     where: {
#       siloAsset: "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5-0x1bea0050e63e05fbb5d8ba2f10cf5800b6224449"
#     }
#     orderBy: season
#     orderDirection: desc
#     first: 10 
#   ) {
#     season
#     siloAsset {
#       totalDepositedAmount
#     	totalDepositedBDV
#     }
#   }
# }